In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

%matplotlib inline
plt.style.use('ggplot')

In [3]:
df = pd.read_csv('athlete_events.csv')
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
df_OLY_all = df[df['Sport'].str.contains('Basketball', regex=True)]
df_OLY_all.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
167,69,Tamara Abalde Daz,F,19.0,185.0,72.0,Spain,ESP,2008 Summer,2008,Summer,Beijing,Basketball,Basketball Women's Basketball,NaN
250,124,Youssef Mohamed Abbas,M,31.0,NaN,NaN,Egypt,EGY,1952 Summer,1952,Summer,Helsinki,Basketball,Basketball Men's Basketball,NaN
264,136,Alessandro Abbio,M,29.0,195.0,85.0,Italy,ITA,2000 Summer,2000,Summer,Sydney,Basketball,Basketball Men's Basketball,NaN
346,192,Ahmed El-Sayed Abdel Hamid Mobarak,M,25.0,189.0,85.0,Egypt,EGY,1972 Summer,1972,Summer,Munich,Basketball,Basketball Men's Basketball,NaN


In [5]:
df_OLY = df_OLY_all[df_OLY_all['Team']=='China']
df_OLY.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
11690,6381,Ba Yan,F,21.0,183.0,78.0,China,CHN,1984 Summer,1984,Summer,Los Angeles,Basketball,Basketball Women's Basketball,Bronze
21295,11239,Bian Lan,F,17.0,180.0,73.0,China,CHN,2004 Summer,2004,Summer,Athina,Basketball,Basketball Women's Basketball,NaN
21296,11239,Bian Lan,F,21.0,180.0,73.0,China,CHN,2008 Summer,2008,Summer,Beijing,Basketball,Basketball Women's Basketball,NaN
39283,20211,Chen Jianghua,M,19.0,187.0,73.0,China,CHN,2008 Summer,2008,Summer,Beijing,Basketball,Basketball Men's Basketball,NaN


In [6]:
df_OLY.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [7]:
df_OLY.loc[:,'Year'] = pd.to_datetime(df_OLY.loc[:,'Year'])
df_OLY.dtypes

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


ID                 int64
Name              object
Sex               object
Age              float64
Height           float64
Weight           float64
Team              object
NOC               object
Games             object
Year      datetime64[ns]
Season            object
City              object
Sport             object
Event             object
Medal             object
dtype: object

In [8]:
df_OLY

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1970-01-01 00:00:00.000001992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
11690,6381,Ba Yan,F,21.0,183.0,78.0,China,CHN,1984 Summer,1970-01-01 00:00:00.000001984,Summer,Los Angeles,Basketball,Basketball Women's Basketball,Bronze
21295,11239,Bian Lan,F,17.0,180.0,73.0,China,CHN,2004 Summer,1970-01-01 00:00:00.000002004,Summer,Athina,Basketball,Basketball Women's Basketball,NaN
21296,11239,Bian Lan,F,21.0,180.0,73.0,China,CHN,2008 Summer,1970-01-01 00:00:00.000002008,Summer,Beijing,Basketball,Basketball Women's Basketball,NaN
39283,20211,Chen Jianghua,M,19.0,187.0,73.0,China,CHN,2008 Summer,1970-01-01 00:00:00.000002008,Summer,Beijing,Basketball,Basketball Men's Basketball,NaN
39284,20211,Chen Jianghua,M,23.0,187.0,73.0,China,CHN,2012 Summer,1970-01-01 00:00:00.000002012,Summer,London,Basketball,Basketball Men's Basketball,NaN
39311,20225,Chen Ke,M,25.0,204.0,104.0,China,CHN,2004 Summer,1970-01-01 00:00:00.000002004,Summer,Athina,Basketball,Basketball Men's Basketball,NaN
39343,20241,Chen Luyun,F,27.0,188.0,73.0,China,CHN,2004 Summer,1970-01-01 00:00:00.000002004,Summer,Athina,Basketball,Basketball Women's Basketball,NaN
39352,20245,Chen Nan,F,21.0,195.0,94.0,China,CHN,2004 Summer,1970-01-01 00:00:00.000002004,Summer,Athina,Basketball,Basketball Women's Basketball,NaN
39353,20245,Chen Nan,F,25.0,195.0,94.0,China,CHN,2008 Summer,1970-01-01 00:00:00.000002008,Summer,Beijing,Basketball,Basketball Women's Basketball,NaN
